__MonkeyPox Analysis Pipeline: Cura Lab__

__Overview__

Mpox (formerly monkeypox) is an infectious disease caused by the mpox virus. It can cause a painful rash, enlarged lymph nodes and fever. Most people fully recover, but some get very sick.

The Mpox virus is an enveloped double-stranded DNA virus of the Orthopoxvirus genus in the Poxviridae family, which includes variola, cowpox, vaccinia and other viruses. The two genetic clades of the virus are clades I and II.

Viruses often mutate to avoid host immune pressure. These mutations sometimes allow viruses to display different traits, such as higher rates of infectivity or transmission; or facilitate worse symptoms in susceptible individuals. Tracking how viruses change geographically and chronologically can help inform epidemiologists, scientists, and physicians how to best respond to outbreaks, and plan future outbreak responses. 

Information taken from: https://www.who.int/news-room/fact-sheets/detail/monkeypox

__Purpose__

In this lab, you will run an analysis and comparison of a recently reported Mpox isolate from the Los Angeles County Public Health Lab microbial pathogen submission group (LACPHL) to an isolate from a recent outbreak in the Canary Islands (https://www.ncbi.nlm.nih.gov/nuccore/ON563414.3?report=fasta). This genome has been used as a reference genome by various groups to quantify and characterize differences between viral isolates post-outbreak. 

__1. Install Dependencies__

In this section, you'll set up your working environment, install all the necessary packages, and clone the necessary repositories for running this analysis and comparison. 

Codespaces has virtual machines available to use when in Jupyter notebooks. First, hover over "select kernel" in the top right. Multiple options should populate, one of which says "install python and jupyter extensions". Click this. Once that is complete, click on "select kernel" again, and then choose "Python Environments". There should be an available Python environment to select. 

Install mamba

In [ ]:
! curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
! bash Mambaforge-$(uname)-$(uname -m).sh -b -p $HOME/mambaforge

Add to your PATH

In [ ]:
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/mambaforge/bin"

In [ ]:
#configure conda channels
! conda config --add channels r
! conda config --add channels bioconda
! conda config --add channels conda-forge

In [ ]:
#install depedencies
! mamba install -y minimap2 samtools lofreq sra-tools fastp
! pip install biopython
! git clone https://github.com/ncbi/sra-human-scrubber

In [ ]:
#make sure the packages are all installed and up to date
! mamba update --all -y
! mamba list

You can see that all of the packages and repositories that we need to run this pipeline are in our environment, or are available in our directory, respectively.

__2. Import Necessary Files__

Follow these instructions to download the reference sequence from GenBank: 



In [ ]:
mkdir reference raw_fastq cleaned_fastq mapped_sorted variant_vcf

In [ ]:
import os
from Bio import SeqIO
from Bio import Entrez
#option to create a list of accession numbers, here we just download one reference genome
acc_nums=['ON563414']

#use the bio.entrez toolkit within biopython to download the accession numbers
#save those sequences to a single fasta file
Entrez.email = "email@example.com"  # Always tell NCBI who you are
filename = "reference/mpox_ON563414.fasta"
if not os.path.isfile(filename):
    # Downloading...
    for acc in acc_nums:
        net_handle = Entrez.efetch(
            db="nucleotide", id=acc, rettype="fasta", retmode="text"
        )
        out_handle = open(filename, "a")
        out_handle.write(net_handle.read())
        out_handle.close()
        net_handle.close()
        print("Saved",acc)


In [ ]:
# make sure the fasta file is present in the reference folder
! ls reference/*

Alternatively, you can manually download like this:
+ Browse to GenBank.
+ Select 'Nucleotide' from the combo box.  
+ Fill in the accession code of the sequence you want to download (i.e. ON563414, found here: https://www.ncbi.nlm.nih.gov/nuccore/ON563414.3?report=fasta) or just write the name of the species (i.e. Monkeypox, and then click on a certain accession code you are interested in).  
+ Click on 'FASTA' link  
+ Click on 'Send to' on the upper right part of the screen.  
+ Select the option 'file'.  
+ Select 'FASTA' as download format.  
+ Click on 'Generate' button. 

Once the sequence is downloaded, right click on your notebook folder, choose "Upload", find your reference sequence.fasta file and upload it. It should appear to the left in your directory.

We are going to download the sra compressed format file directly from AWS using [this link](https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR23893269&display=data-access)

In [ ]:
! wget https://sra-pub-run-odp.s3.amazonaws.com/sra/SRR23893269/SRR23893269 --no-check-certificate

In [ ]:
#convert the compressed format to Fastq
! fasterq-dump -O raw_fastq SRR23893269

You can see that we now have three fasta/q files in our directory: a reference sequence, and two fastq sequences from the isolate.

In [ ]:
#print the first few lines of each file for inspection
! head -20 reference/mpox_ON563414.fasta 
! head -10 raw_fastq/SRR23893269_1.fastq
! head -10 raw_fastq/SRR23893269_1.fastq


You can see above the difference between the fasta (first file) and the two fastq (bottom two) files. The fasta file has some identifying information and then the raw sequence information, and the two fastq files include the sequence information with accompanying quality information about the sequences. 

__3. FASTQ Aggregation and Interleaving Using FASTP__

In this step, you use the FASTP package to perform quality control and filtering of the reads in both files, and then interleaving (combining) of the two into one file. 

In [ ]:
#uses fastp to merge the two input fastq files of the variant while simultaneously filtering the reads for quality control
! fastp --merge --in1 raw_fastq/SRR23893269_1.fastq --in2 raw_fastq/SRR23893269_2.fastq --merged_out cleaned_fastq/SRR23893269_cleaned_interleaved.fastq

__4. Human Reads Scrubbing__

Because you are aligning and analyzing viral genomes, a good step in this particular pipeline is to perform "scrubbing" of your isolate files to make sure that there are no human reads mixed in with the viral genome reads. This step checks and cleanses your files of human reads. 

In [ ]:
%%bash
#pre-processing command to add in a human reads database to check for the human reads
cd /workspaces/DeloitteBioinformatics/sra-human-scrubber
/workspaces/DeloitteBioinformatics/sra-human-scrubber/init_db.sh

In [ ]:
%%bash
#change directories to access the scrubbing script and run it on your interleaved fastq file
cd /workspaces/DeloitteBioinformatics/sra-human-scrubber/scripts
/workspaces/DeloitteBioinformatics/sra-human-scrubber/scripts/scrub.sh -i /workspaces/DeloitteBioinformatics/cleaned_fastq/SRR23893269_cleaned_interleaved.fastq -o /workspaces/DeloitteBioinformatics/cleaned_fastq/cleaned_interleaved_scrubbed.fastq

__5. Mappying Reads to the Reference Genome__

Now that the reads are quality controlled, interleaved, and scrubbed on human reads, you can align your isolate fastq to your reference genome.

In [ ]:
#mapping non-human reads to the reference sequence using minimap2
! minimap2 -ax sr reference/mpox_ON563414.fasta cleaned_fastq/cleaned_interleaved_scrubbed.fastq > mapped_sorted/aligned.sam

__6. Sort and Index Aligned Reads__

In this step, you sort and index your aligned reads using a package called samtools.

In [ ]:
#sort and index aligned reads
! samtools sort mapped_sorted/aligned.sam > mapped_sorted/aligned_sorted.bam

In [ ]:
#discard unmapped reads from sorted BAM file
! samtools view -F 0x04 -b mapped_sorted/aligned_sorted.bam > mapped_sorted/aligned_sorted_mapped.bam

In [ ]:
#index BAM file
! samtools index mapped_sorted/aligned_sorted_mapped.bam

__7. Variant Calling__

In this last step, we will call all of the variants found in our isolate file and inspect them in comparison to the reference file.

In [ ]:
%%bash
#variant calling
lofreq call -f reference/mpox_ON563414.fasta -o variant_vcf/variants.vcf mapped_sorted/aligned_sorted_mapped.bam

In [ ]:
! head -50 variant_vcf/variants.vcf

You can see that there are quite a few changes between the isolate we used and the 'reference' genome from a previous outbreak (comparing the entries in the REF and ALT column, columns 4 and 5 respectively).

__Conclusion and References__

In this lab, you processed some publically available MNPX fastq files and compared the sequence of this isolate to another publically available MNPX genome from a previous outbreak in 2022. Great job!

This pipeline was built utilizing this MNPX analysis as a framework: https://github.com/genomicsITER/monkeypox. 

__Troubleshooting__

You should not need to install conda, as the python environment that comes with codespaces should already have it installed. In instances where you need to install it, please use the following code (will also work if you are trying to run this notebook in another platform, like Google Colab).

In [ ]:
#install miniconda
!MINICONDA_INSTALLER_SCRIPT=Miniconda3-latest-Linux-x86_64.sh
!MINICONDA_PREFIX=/usr/local
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x $MINICONDA_INSTALLER_SCRIPT
!./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX